In [76]:
# Import and set up db
import sqlalchemy as db
engine = db.create_engine('sqlite:///app.db')
connection = engine.connect()
metadata = db.MetaData()

In [77]:
# Card table
card = db.Table('Card', metadata, autoload_with=engine)
# Commander Collection table
cc = db.Table('Commander_collection', metadata, autoload_with=engine)

In [78]:
card.columns.keys()

['id',
 'card_name',
 'search_name',
 'oracle_text',
 'type_line',
 'power',
 'toughness',
 'mana_cost',
 'cmc',
 'color_identity',
 'card_img',
 'double_sided_pointer']

In [79]:
cc.columns.keys()

['id', 'commander_id', 'card_id']

In [179]:
query = db.select(card.c.get('oracle_text')).select_from(card).join(cc, card.c.id == cc.c.card_id).where(cc.c.commander_id == 1)

In [180]:
# This is now a list of oricle text... Kinda. The format is messy.
ugly_ob_list = connection.execute(query).scalars().all()

In [181]:
import re
import spacy

In [182]:
# Get rid of computer formatting stuff.
cleaned_suggestions = []
for card_inst in ugly_ob_list:
    cleaned_suggestions.append(card_inst.replace('\n',' ').strip())
ob_body = ' '.join(cleaned_suggestions)

In [183]:
# Get rid of stop words
ob_body = re.sub(r' on | the | of | to | and | with | in ',' ',ob_body)

In [184]:
# print(ob_body)

In [185]:
npl = spacy.load("en_core_web_sm")

In [186]:
doc = npl(ob_body)

In [187]:
lemmas = []
for token in doc:
    if not token.is_punct:
        lemmas.append(token.lemma_)

In [188]:
lemetized_ob = ' '.join(lemmas)

In [189]:
# print(lemetized_ob)

In [190]:
from sklearn.feature_extraction.text import CountVectorizer

In [191]:
v = CountVectorizer(ngram_range=(2,3), token_pattern=r'(?u)\b[\d\w]+\b')
v.fit([lemetized_ob])
vocab = v.vocabulary_

In [192]:
# print(vocab)

In [197]:
for phrase in vocab:
    count = lemetized_ob.count(phrase)
    if count > 40 and count < 100:
        print(phrase+': ', count)

you may:  68
deal 1:  77
1 damage:  78
t add:  76
1 life:  51
r t:  50
a card:  51
enter battlefield:  61
you control:  52
your library:  52
whenever a:  41
r you:  47
battlefield t:  44
or a:  43
card t:  41
deal 1 damage:  76
